In [9]:
import pandas as pd
from sqlalchemy import create_engine
from datetime import datetime, timedelta

# Database credentials
db_username = 'admin'
db_password = 'UBCParking2024'
db_server = 'testdb.cdq6s8s6klpd.ca-central-1.rds.amazonaws.com'
db_name = 'Parking'

# SQLAlchemy connection string
connection_string = f'mssql+pyodbc://{db_username}:{db_password}@{db_server}/{db_name}?driver=ODBC+Driver+17+for+SQL+Server'

# Create the SQLAlchemy engine
engine = create_engine(connection_string)

# List of tables to query with their new names
tables = {
    'dbo.NorthParkade_Occupancy': 'North',
    'dbo.WestParkade_Occupancy': 'West',
    'dbo.RoseGardenParkade_Occupancy': 'Rose',
    'dbo.HealthSciencesParkade_Occupancy': 'Health Sciences',
    'dbo.FraserParkade_Occupancy': 'Fraser',
    'dbo.ThunderbirdParkade_Occupancy': 'Thunderbird',
    'dbo.UnivWstBlvdParkade_Occupancy': 'University Lot Blvd'
}

# Define the timestamp to filter rows
timestamp_cutoff = '2024-03-05 23:00:00'
# Convert timestamp_cutoff to Unix timestamp
timestamp_unix = int(datetime.strptime(timestamp_cutoff, '%Y-%m-%d %H:%M:%S').timestamp())

# Initialize DataFrames to hold results
all_data = pd.DataFrame()
all_data_by_min = pd.DataFrame()

# List to keep track of missing times
missing_times = []

# Query each table and concatenate results
with engine.connect() as connection:
    for table, new_name in tables.items():
        print(f"Querying table: {table}")
        
        query = f"""
        SELECT Vehicles, TimestampUnix
        FROM {table}
        """
        df = pd.read_sql(query, connection)
        
        # Print the raw data retrieved
        print(f"Raw data from {table}:")
        print(df.head())
        
        # Convert TimestampUnix to datetime
        df['Timestamp'] = pd.to_datetime(df['TimestampUnix'], unit='s')
        
        # Group by minute for all data by minute
        df['Minute'] = df['Timestamp'].dt.floor('min')
        df_minute = df.groupby('Minute').last().reset_index()
        
        df_minute = df_minute.rename(columns={'Vehicles': new_name})
        
        # Generate a list of expected hours and minutes
        start_time = df['Timestamp'].min().floor('H')
        end_time = df['Timestamp'].max().ceil('H')
        all_minutes = pd.date_range(start=start_time, end=end_time, freq='T')

        missing_minutes = all_minutes.difference(df_minute['Minute'])
        
        missing_times.extend(missing_minutes)
        
        if all_data_by_min.empty:
            all_data_by_min = df_minute[['Minute', new_name]]
        else:
            all_data_by_min = pd.merge(all_data_by_min, df_minute[['Minute', new_name]], on='Minute', how='outer')
        
        # Print merged data so far
        print(f"Merged data by hour so far:")
        print(all_data.head())
        print(f"Merged data by minute so far:")
        print(all_data_by_min.head())

# Save the results to CSV files
csv_file_hourly = 'output_hourly.csv'
csv_file_minute = 'output_minute.csv'
missing_times_file = 'missing_times.csv'

all_data.to_csv(csv_file_hourly, index=False)
all_data_by_min.to_csv(csv_file_minute, index=False)

# Save the list of missing times to a CSV file
missing_times_df = pd.DataFrame(missing_times, columns=['MissingTimes'])
missing_times_df.to_csv(missing_times_file, index=False)

print(f"Hourly data has been successfully saved to {csv_file_hourly}")
print(f"Minute data has been successfully saved to {csv_file_minute}")
print(f"Missing times have been successfully saved to {missing_times_file}")


Querying table: dbo.NorthParkade_Occupancy
Raw data from dbo.NorthParkade_Occupancy:
   Vehicles  TimestampUnix
0        90     1645901520
1        89     1645901580
2        89     1645901640
3        89     1645901700
4        90     1645901760


C:\Users\ginap\AppData\Local\Temp\ipykernel_5884\4117907214.py:59: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df['Hour'] = df['Timestamp'].dt.floor('H')
C:\Users\ginap\AppData\Local\Temp\ipykernel_5884\4117907214.py:73: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  start_time = df['Timestamp'].min().floor('H')
C:\Users\ginap\AppData\Local\Temp\ipykernel_5884\4117907214.py:74: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  end_time = df['Timestamp'].max().ceil('H')
C:\Users\ginap\AppData\Local\Temp\ipykernel_5884\4117907214.py:75: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  all_hours = pd.date_range(start=start_time, end=end_time, freq='H')
C:\Users\ginap\AppData\Local\Temp\ipykernel_5884\4117907214.py:76: FutureWarning: 'T' is deprecated and will be removed in a future 

Merged data by hour so far:
                 Hour  North
0 2018-09-06 09:00:00    950
1 2018-09-06 10:00:00    918
2 2018-09-06 11:00:00    832
3 2018-09-06 12:00:00    850
4 2018-09-06 13:00:00    859
Merged data by minute so far:
               Minute  North
0 2018-09-06 09:28:00    890
1 2018-09-06 09:29:00    886
2 2018-09-06 09:30:00    886
3 2018-09-06 09:31:00    894
4 2018-09-06 09:32:00    895
Querying table: dbo.WestParkade_Occupancy
Raw data from dbo.WestParkade_Occupancy:
   Vehicles  TimestampUnix
0       470     1717704480
1       158     1717720320
2        41     1717738380
3        20     1717741380
4        20     1717741440


C:\Users\ginap\AppData\Local\Temp\ipykernel_5884\4117907214.py:59: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df['Hour'] = df['Timestamp'].dt.floor('H')
C:\Users\ginap\AppData\Local\Temp\ipykernel_5884\4117907214.py:73: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  start_time = df['Timestamp'].min().floor('H')
C:\Users\ginap\AppData\Local\Temp\ipykernel_5884\4117907214.py:74: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  end_time = df['Timestamp'].max().ceil('H')
C:\Users\ginap\AppData\Local\Temp\ipykernel_5884\4117907214.py:75: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  all_hours = pd.date_range(start=start_time, end=end_time, freq='H')
C:\Users\ginap\AppData\Local\Temp\ipykernel_5884\4117907214.py:76: FutureWarning: 'T' is deprecated and will be removed in a future 

Merged data by hour so far:
                 Hour  North    West
0 2018-09-06 09:00:00    950  1015.0
1 2018-09-06 10:00:00    918  1160.0
2 2018-09-06 11:00:00    832  1210.0
3 2018-09-06 12:00:00    850  1166.0
4 2018-09-06 13:00:00    859  1219.0
Merged data by minute so far:
               Minute  North   West
0 2018-09-06 09:28:00  890.0    NaN
1 2018-09-06 09:29:00  886.0  929.0
2 2018-09-06 09:30:00  886.0  936.0
3 2018-09-06 09:31:00  894.0  942.0
4 2018-09-06 09:32:00  895.0  945.0
Querying table: dbo.RoseGardenParkade_Occupancy
Raw data from dbo.RoseGardenParkade_Occupancy:
   Vehicles  TimestampUnix
0       100     1633285140
1        98     1633285200
2        98     1633285260
3        97     1633285320
4        94     1633285380


...

Merged data by hour so far:
                 Hour  North    West  Rose  Health Sciences  Fraser
0 2018-09-06 09:00:00    950  1015.0   475              909     556
1 2018-09-06 10:00:00    918  1160.0   612              972     706
2 2018-09-06 11:00:00    832  1210.0   659              964     724
3 2018-09-06 12:00:00    850  1166.0   695              953     720
4 2018-09-06 13:00:00    859  1219.0   709              962     727
Merged data by minute so far:
               Minute  North   West   Rose  Health Sciences  Fraser
0 2018-09-06 09:28:00  890.0    NaN  406.0              NaN     NaN
1 2018-09-06 09:29:00  886.0  929.0  404.0            828.0   499.0
2 2018-09-06 09:30:00  886.0  936.0  412.0            832.0   501.0
3 2018-09-06 09:31:00  894.0  942.0  416.0            837.0   503.0
4 2018-09-06 09:32:00  895.0  945.0  418.0            837.0   504.0
Querying table: dbo.ThunderbirdParkade_Occupancy
Raw data from dbo.ThunderbirdParkade_Occupancy:
   Vehicles  TimestampUnix
0 

C:\Users\ginap\AppData\Local\Temp\ipykernel_5884\4117907214.py:59: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df['Hour'] = df['Timestamp'].dt.floor('H')
C:\Users\ginap\AppData\Local\Temp\ipykernel_5884\4117907214.py:73: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  start_time = df['Timestamp'].min().floor('H')
C:\Users\ginap\AppData\Local\Temp\ipykernel_5884\4117907214.py:74: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  end_time = df['Timestamp'].max().ceil('H')
C:\Users\ginap\AppData\Local\Temp\ipykernel_5884\4117907214.py:75: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  all_hours = pd.date_range(start=start_time, end=end_time, freq='H')
C:\Users\ginap\AppData\Local\Temp\ipykernel_5884\4117907214.py:76: FutureWarning: 'T' is deprecated and will be removed in a future 

Merged data by hour so far:
                 Hour  North    West  Rose  Health Sciences  Fraser  \
0 2018-09-06 09:00:00    950  1015.0   475              909     556   
1 2018-09-06 10:00:00    918  1160.0   612              972     706   
2 2018-09-06 11:00:00    832  1210.0   659              964     724   
3 2018-09-06 12:00:00    850  1166.0   695              953     720   
4 2018-09-06 13:00:00    859  1219.0   709              962     727   

   Thunderbird  
0          934  
1         1049  
2         1072  
3         1095  
4         1073  
Merged data by minute so far:
               Minute  North   West   Rose  Health Sciences  Fraser  \
0 2018-09-06 09:28:00  890.0    NaN  406.0              NaN     NaN   
1 2018-09-06 09:29:00  886.0  929.0  404.0            828.0   499.0   
2 2018-09-06 09:30:00  886.0  936.0  412.0            832.0   501.0   
3 2018-09-06 09:31:00  894.0  942.0  416.0            837.0   503.0   
4 2018-09-06 09:32:00  895.0  945.0  418.0            837.

C:\Users\ginap\AppData\Local\Temp\ipykernel_5884\4117907214.py:59: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df['Hour'] = df['Timestamp'].dt.floor('H')
C:\Users\ginap\AppData\Local\Temp\ipykernel_5884\4117907214.py:73: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  start_time = df['Timestamp'].min().floor('H')
C:\Users\ginap\AppData\Local\Temp\ipykernel_5884\4117907214.py:74: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  end_time = df['Timestamp'].max().ceil('H')
C:\Users\ginap\AppData\Local\Temp\ipykernel_5884\4117907214.py:75: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  all_hours = pd.date_range(start=start_time, end=end_time, freq='H')
C:\Users\ginap\AppData\Local\Temp\ipykernel_5884\4117907214.py:76: FutureWarning: 'T' is deprecated and will be removed in a future 

Merged data by hour so far:
                 Hour  North    West  Rose  Health Sciences  Fraser  \
0 2018-09-06 09:00:00    950  1015.0   475              909     556   
1 2018-09-06 10:00:00    918  1160.0   612              972     706   
2 2018-09-06 11:00:00    832  1210.0   659              964     724   
3 2018-09-06 12:00:00    850  1166.0   695              953     720   
4 2018-09-06 13:00:00    859  1219.0   709              962     727   

   Thunderbird  University Lot Blvd  
0          934                  NaN  
1         1049                  NaN  
2         1072                  NaN  
3         1095                  NaN  
4         1073                  NaN  
Merged data by minute so far:
               Minute  North   West   Rose  Health Sciences  Fraser  \
0 2018-09-06 09:28:00  890.0    NaN  406.0              NaN     NaN   
1 2018-09-06 09:29:00  886.0  929.0  404.0            828.0   499.0   
2 2018-09-06 09:30:00  886.0  936.0  412.0            832.0   501.0   
3 2

In [11]:
import pandas as pd

# Sample data (replace this with your actual data)
df = pd.DataFrame(all_data)

# Set Hour as index
df = df.set_index('Hour')

# Generate expected hours
start_hour = df.index.min()
end_hour = df.index.max()
expected_hours = pd.date_range(start=start_hour, end=end_hour, freq='H')

# Reindex to fill missing hours with zeros
df_filled = df.reindex(expected_hours, fill_value=0)

# Identify gaps in the data
for column in df_filled.columns:
    gap_indices = df_filled[df_filled[column] == 0].index
    for idx in gap_indices:
        next_value = df_filled.loc[idx:].loc[df_filled[column] != 0].iloc[0][column]
        df_filled.loc[idx, column] = next_value

# Reset index
df_filled = df_filled.reset_index()

print(df_filled)


C:\Users\ginap\AppData\Local\Temp\ipykernel_5884\2770567029.py:12: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  expected_hours = pd.date_range(start=start_hour, end=end_hour, freq='H')


                    index  North    West  Rose  Health Sciences  Fraser  \
0     2018-09-06 09:00:00    950  1015.0   475              909     556   
1     2018-09-06 10:00:00    918  1160.0   612              972     706   
2     2018-09-06 11:00:00    832  1210.0   659              964     724   
3     2018-09-06 12:00:00    850  1166.0   695              953     720   
4     2018-09-06 13:00:00    859  1219.0   709              962     727   
...                   ...    ...     ...   ...              ...     ...   
51198 2024-07-09 15:00:00    445   349.0    89              562      93   
51199 2024-07-09 16:00:00    546   493.0   206              688     180   
51200 2024-07-09 17:00:00    585   509.0   237              741     201   
51201 2024-07-09 18:00:00    583   569.0   249              742     200   
51202 2024-07-09 19:00:00    592   570.0   250              731     202   

       Thunderbird  University Lot Blvd  
0              934                  NaN  
1             1

In [13]:
df_filled['Hour'] = df_filled['index']
df_filled = df_filled.drop('index', axis=1)

In [14]:
df_filled.to_csv("transposed_output_hourly.csv")

In [ ]:
'''
# Fill missing hours with 0
#all_data['Hour'] = all_data['index']
start_hour = all_data['Hour'].min()
end_hour = all_data['Hour'].max()
expected_hours = pd.date_range(start=start_hour, end=end_hour, freq='H')
all_data = all_data.set_index('Hour').reindex(expected_hours).fillna(0).reset_index()
all_data['Hour'] = all_data['index']
all_data['Hour'] = all_data['Hour'].dt.strftime('%Y-%m-%d %H:%M:%S')
all_data
#all_data = all_data.drop('level_0', axis=1)
all_data = all_data.drop('index', axis=1)

# Save the results to CSV files
csv_file_hourly = 'output_hourly.csv'
csv_file_minute = 'output_minute.csv'

all_data.to_csv(csv_file_hourly, index=False)
all_data_by_min.to_csv(csv_file_minute, index=False)

print(f"Hourly data has been successfully saved to {csv_file_hourly}")
print(f"Minute data has been successfully saved to {csv_file_minute}")

'''



C:\Users\ginap\AppData\Local\Temp\ipykernel_5884\2367192603.py:5: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  expected_hours = pd.date_range(start=start_hour, end=end_hour, freq='H')


Hourly data has been successfully saved to output_hourly.csv
Minute data has been successfully saved to output_minute.csv


In [ ]:
all_data

,North,West,Rose,Health Sciences,Fraser,Thunderbird,University Lot Blvd,Hour
0,950.0,1015.0,475.0,909.0,556.0,934.0,0.0,2018-09-06 09:00:00
1,918.0,1160.0,612.0,972.0,706.0,1049.0,0.0,2018-09-06 10:00:00
2,832.0,1210.0,659.0,964.0,724.0,1072.0,0.0,2018-09-06 11:00:00
3,850.0,1166.0,695.0,953.0,720.0,1095.0,0.0,2018-09-06 12:00:00
4,859.0,1219.0,709.0,962.0,727.0,1073.0,0.0,2018-09-06 13:00:00
...,...,...,...,...,...,...,...,...
51198,445.0,349.0,89.0,562.0,93.0,343.0,39.0,2024-07-09 15:00:00
51199,546.0,493.0,206.0,688.0,180.0,484.0,43.0,2024-07-09 16:00:00
51200,585.0,509.0,237.0,741.0,201.0,533.0,44.0,2024-07-09 17:00:00
51201,583.0,569.0,249.0,742.0,200.0,533.0,78.0,2024-07-09 18:00:00


In [ ]:
import pandas as pd
from sqlalchemy import create_engine
from datetime import datetime, timedelta

# Database credentials
db_username = 'admin'
db_password = 'UBCParking2024'
db_server = 'testdb.cdq6s8s6klpd.ca-central-1.rds.amazonaws.com'
db_name = 'Parking'

# SQLAlchemy connection string
connection_string = f'mssql+pyodbc://{db_username}:{db_password}@{db_server}/{db_name}?driver=ODBC+Driver+17+for+SQL+Server'

# Create the SQLAlchemy engine
engine = create_engine(connection_string)

# List of tables to query with their new names
tables = {
    'dbo.NorthParkade_Occupancy': 'North',
    'dbo.WestParkade_Occupancy': 'West',
    'dbo.RoseGardenParkade_Occupancy': 'Rose',
    'dbo.HealthSciencesParkade_Occupancy': 'Health Sciences',
    'dbo.FraserParkade_Occupancy': 'Fraser',
    'dbo.ThunderbirdParkade_Occupancy': 'Thunderbird',
    'dbo.UnivWstBlvdParkade_Occupancy': 'University Lot Blvd'
}

# Define the timestamp to filter rows
timestamp_cutoff = '2024-03-05 23:00:00'
# Convert timestamp_cutoff to Unix timestamp
timestamp_unix = int(datetime.strptime(timestamp_cutoff, '%Y-%m-%d %H:%M:%S').timestamp())

# Initialize DataFrames to hold results
all_data = pd.DataFrame()
all_data_by_min = pd.DataFrame()

# Query each table and concatenate results
with engine.connect() as connection:
    for table, new_name in tables.items():
        print(f"Querying table: {table}")
        
        query = f"""
        SELECT Vehicles, TimestampUnix
        FROM {table}
        """
        df = pd.read_sql(query, connection)
        
        # Convert TimestampUnix to datetime
        df['Timestamp'] = pd.to_datetime(df['TimestampUnix'], unit='s')
        
        # Round down to the nearest hour for aggregation by hour
        df['Hour'] = df['Timestamp'].dt.floor('H')
        
        # Group by hour and take the latest record for each hour
        df_hourly = df.groupby('Hour').last().reset_index()
        
        # Rename the Vehicles column to the new parkade name
        df_hourly = df_hourly.rename(columns={'Vehicles': new_name})
        
        # Merge data into aggregated DataFrames
        if all_data.empty:
            all_data = df_hourly[['Hour', new_name]]
        else:
            all_data = pd.merge(all_data, df_hourly[['Hour', new_name]], on='Hour', how='outer')
        
        # Print merged data so far
        print(f"Merged data by hour so far:")
        print(all_data.head())
        
# Fill missing hours and shift data accordingly
all_data = all_data.sort_values(by='Hour').reset_index(drop=True)
all_data_filled = all_data.copy()

prev_hour = None
for idx, row in all_data.iterrows():
    current_hour = row['Hour']
    if prev_hour is not None:
        while prev_hour + timedelta(hours=1) < current_hour:
            missing_hour = prev_hour + timedelta(hours=1)
            print(f"Inserting missing hour: {missing_hour}")
            # Insert a row with NaNs for all parkades for the missing hour
            new_row = {'Hour': missing_hour}
            for new_name in tables.values():
                new_row[new_name] = None
            all_data_filled = all_data_filled.append(new_row, ignore_index=True)
            # Shift data after the gap
            all_data_filled.loc[all_data_filled['Hour'] > missing_hour, all_data_filled.columns != 'Hour'] = \
                all_data_filled.loc[all_data_filled['Hour'] > missing_hour, all_data_filled.columns != 'Hour'].shift(-1)
    
    prev_hour = current_hour

# Save the results to a CSV file
csv_file_hourly = 'output_hourly.csv'
all_data_filled.to_csv(csv_file_hourly, index=False)

print(f"Hourly data has been successfully saved to {csv_file_hourly}")


Querying table: dbo.NorthParkade_Occupancy


C:\Users\ginap\AppData\Local\Temp\ipykernel_24032\1523829458.py:52: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df['Hour'] = df['Timestamp'].dt.floor('H')


Merged data by hour so far:
                 Hour  North
0 2018-09-06 09:00:00    950
1 2018-09-06 10:00:00    918
2 2018-09-06 11:00:00    832
3 2018-09-06 12:00:00    850
4 2018-09-06 13:00:00    859
Querying table: dbo.WestParkade_Occupancy


C:\Users\ginap\AppData\Local\Temp\ipykernel_24032\1523829458.py:52: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df['Hour'] = df['Timestamp'].dt.floor('H')


Merged data by hour so far:
                 Hour  North    West
0 2018-09-06 09:00:00    950  1015.0
1 2018-09-06 10:00:00    918  1160.0
2 2018-09-06 11:00:00    832  1210.0
3 2018-09-06 12:00:00    850  1166.0
4 2018-09-06 13:00:00    859  1219.0
Querying table: dbo.RoseGardenParkade_Occupancy


...

C:\Users\ginap\AppData\Local\Temp\ipykernel_24032\1523829458.py:52: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df['Hour'] = df['Timestamp'].dt.floor('H')


Merged data by hour so far:
                 Hour  North    West  Rose  Health Sciences  Fraser  \
0 2018-09-06 09:00:00    950  1015.0   475              909     556   
1 2018-09-06 10:00:00    918  1160.0   612              972     706   
2 2018-09-06 11:00:00    832  1210.0   659              964     724   
3 2018-09-06 12:00:00    850  1166.0   695              953     720   
4 2018-09-06 13:00:00    859  1219.0   709              962     727   

   Thunderbird  
0          934  
1         1049  
2         1072  
3         1095  
4         1073  
Querying table: dbo.UnivWstBlvdParkade_Occupancy


C:\Users\ginap\AppData\Local\Temp\ipykernel_24032\1523829458.py:52: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df['Hour'] = df['Timestamp'].dt.floor('H')


Merged data by hour so far:
                 Hour  North    West  Rose  Health Sciences  Fraser  \
0 2018-09-06 09:00:00    950  1015.0   475              909     556   
1 2018-09-06 10:00:00    918  1160.0   612              972     706   
2 2018-09-06 11:00:00    832  1210.0   659              964     724   
3 2018-09-06 12:00:00    850  1166.0   695              953     720   
4 2018-09-06 13:00:00    859  1219.0   709              962     727   

   Thunderbird  University Lot Blvd  
0          934                  NaN  
1         1049                  NaN  
2         1072                  NaN  
3         1095                  NaN  
4         1073                  NaN  
Inserting missing hour: 2019-03-10 02:00:00


AttributeError: 'DataFrame' object has no attribute 'append'

In [ ]:
import pandas as pd
from sqlalchemy import create_engine

# Database credentials
db_username = 'admin'
db_password = 'UBCParking2024'
db_server = 'testdb.cdq6s8s6klpd.ca-central-1.rds.amazonaws.com'
db_name = 'Parking'

# SQLAlchemy connection string
connection_string = f'mssql+pyodbc://{db_username}:{db_password}@{db_server}/{db_name}?driver=ODBC+Driver+17+for+SQL+Server'

# Create the SQLAlchemy engine
engine = create_engine(connection_string)

# Define the timestamp to filter rows
timestamp_cutoff = '2024-03-05 23:00:00'
timestamp_cutoff_unix = pd.Timestamp(timestamp_cutoff).timestamp()

# Columns to select from the weather table
weather_columns = [
    'temp', 'visibility', 'dew_point', 'feels_like', 
    'pressure', 'wind_speed', 
    'clouds', 'rain', 'snow'
] # we're missing humidity, clouds_all, rain_1h, snow_1h from the db

# Query the actual_weather table
with engine.connect() as connection:
    query = f"""
    SELECT dt, {', '.join(weather_columns)}
    FROM dbo.actual_weather
    ORDER BY dt ASC
    """
    weather_df = pd.read_sql(query, connection)

# Convert Unix timestamp to datetime
weather_df['dt'] = pd.to_datetime(weather_df['dt'], unit='s')
weather_df = weather_df.rename(columns={'dt': 'Timestamp'})

# Print the first few rows of the weather data
print("Weather data retrieved:")
print(weather_df.head())
print(weather_df.tail())

# Save the weather data to a CSV file
weather_csv_file_path = 'weather_output.csv'
weather_df.to_csv(weather_csv_file_path, index=False)

print(f"Weather data has been successfully saved to {weather_csv_file_path}")


Weather data retrieved:
            Timestamp    temp  visibility  dew_point  feels_like  pressure  \
0 2024-05-26 19:00:00  283.01       10000     282.40      279.58    1019.0   
1 2024-05-26 20:00:00  283.85       10000     283.08      283.46    1019.0   
2 2024-05-26 21:00:00  284.24       10000     283.31      283.86    1020.0   
3 2024-05-26 22:00:00  284.52       10000     283.43      284.14    1020.0   
4 2024-05-26 23:00:00  284.48       10000     283.39      284.10    1020.0   

   wind_speed  clouds  rain  snow  
0        8.23     100   0.0   0.0  
1        7.72     100   0.0   0.0  
2       10.29     100   0.0   0.0  
3       10.29     100   0.0   0.0  
4        9.26     100   0.0   0.0  
              Timestamp    temp  visibility  dew_point  feels_like  pressure  \
872 2024-07-02 03:00:00  292.60       10000     285.62      292.27    1019.0   
873 2024-07-02 04:00:00  291.59       10000     285.36      291.24    1019.0   
874 2024-07-02 05:00:00  290.51       10000     284